# Simulating and Evaluating Multiturn Conversations for Content Harms

## Objective

This notebook walks through how to generate a simulated multi-turn conversation targeting a deployed AzureOpenAI model and then evaluate that test dataset for Content Safety harms. 

## Time
You should expect to spend about 30 minutes running this notebook. If you increase or decrease the number of simulated conversations, the time will vary accordingly.

## Before you begin

### Installation
Install the following packages required to execute this notebook.

In [ ]:
%pip install openai azure-ai-evaluation azure-identity promptflow-azure

### Configuration
The following simulator and evaluators require an Azure AI Studio project configuration and an Azure credential to use. 
Your project configuration will be what is used to log your evaluation results in your project after the evaluation run is finished.

For full region supportability, see [our documentation](https://learn.microsoft.com/azure/ai-studio/how-to/develop/flow-evaluate-sdk#built-in-evaluators).

Set the following variables for use in this notebook:

In [ ]:
azure_ai_project = {
    "subscription_id": "<your-subscription-id>",
    "resource_group": "<your-resource-group>",
    "workspace_name": "<your-workspace-name>",
}


azure_openai_endpoint = "<your-azure-openai-endpoint>"
azure_openai_deployment = "<your-deployment-name>"
azure_openai_api_version = "<your-deployment's-api-version>"

In [ ]:
import os

os.environ["AZURE_DEPLOYMENT_NAME"] = azure_openai_deployment
os.environ["AZURE_API_VERSION"] = azure_openai_api_version
os.environ["AZURE_ENDPOINT"] = azure_openai_endpoint

## Run this example

To keep this notebook lightweight, let's create a dummy application that calls an AzureOpenAI model, such as GPT 4. When we are testing your application for certain safety metrics like Content Safety, it's important to have a way to automate a basic style of red-teaming to elicit behaviors from a simulated malicious user. We will use the `Simulator` class and this is how we will generate a synthetic test dataset against your application. Once we have the test dataset, we can evaluate them with our `ContentSafetyEvaluator` class.

The `Simulator` needs a structured contract with your application in order to simulate conversations or other types of interactions with it. This is achieved via a callback function. This is the function you would rewrite to actually format the response from your generative AI application.

In [ ]:
from typing import List, Dict, Optional

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import ContentSafetyEvaluator
from azure.ai.evaluation.simulator import AdversarialSimulator, AdversarialScenario
from openai import AzureOpenAI

credential = DefaultAzureCredential()


async def content_safety_callback(
    messages: List[Dict], stream: bool = False, session_state: Optional[str] = None, context: Optional[Dict] = None
) -> dict:
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")
    endpoint = os.environ.get("AZURE_ENDPOINT")
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    # Get a client handle for the model
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )
    # Call the model
    try:
        completion = client.chat.completions.create(
            model=deployment,
            messages=[
                {
                    "role": "user",
                    "content": messages["messages"][0]["content"],
                }
            ],
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
        )
        formatted_response = completion.to_dict()["choices"][0]["message"]
    except Exception:
        formatted_response = {
            "content": "I don't know",
            "role": "assistant",
            "context": {"key": {}},
        }
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context,
    }

## Testing your application for Content Safety

When building your application, you want to test that Content Safety harms (i.e. Hate and unfairness, Sexual, Violent, Self-harm) are not being generated by your generative AI applications. The following example uses an `AdversarialSimulator` paired with a conversation scenario to prompt your model to respond with material that contains content safety harms.

In [ ]:
content_safety_simulator = AdversarialSimulator(azure_ai_project=azure_ai_project, credential=credential)

content_safety_scenario = AdversarialScenario.ADVERSARIAL_CONVERSATION

Below we explicitly request that the conversation has multiple turns between the User and Assistant. 

In [ ]:
content_safety_outputs = await content_safety_simulator(
    scenario=content_safety_scenario,
    max_conversation_turns=5,  # define the number of conversation turns
    max_simulation_results=5,  # define the number of simulation results
    target=content_safety_callback,  # define the target model callback
)

In [ ]:
import json
from pathlib import Path

with Path("adv_convo_eval.jsonl").open("w") as f:
    for output in content_safety_outputs:
        f.write(json.dumps({"conversation": output}))
        f.write("\n")

Now that we have our dataset, we can evaluate it for Content Safety harms. The `ContentSafetyEvaluator` class can take in the dataset and detect whether your data contains harmful content. Let's use the `evaluate()` API to run the evaluation and log it to our Azure AI Studio Project.

In [ ]:
cs_eval = ContentSafetyEvaluator(azure_ai_project=azure_ai_project, credential=credential)

result = evaluate(
    name="content-safety-conversation",
    data="adv_convo_eval.jsonl",
    evaluators={"content_safety": cs_eval},
    # Optionally provide your AI Studio project information to track your evaluation results in your Azure AI Studio project
    azure_ai_project=azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and studio URL
    output_path="./content-safety-conversation_results.json",
)